# Working with Tools
> Tools are nothing just adding our custom function to LLM's model
> WHY: As we know LLM are very good with strings data in simple words very good in the conversation but it lacks the math calculation and so many other things to overcome this we usally add/provide tools to our model so whenever needed they can use this tool to solve the query of the users
---
> Important points
- It allows frontier model to connect with external functions
    - Richer response by extending knowledge
    - Ability to carry to out outcomes within the application
    - Enhanced capabilities, like calculations

> How it works
- In a request to the LLM, specify available tools
- The reply is either text, or a request to run a tool

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

c:\Users\user\Desktop\Git-hub\LLM-engieering\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Getting api key 
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')
if api_key:
    print(f"API key has been configured... Successfully {api_key[:10]}")
else:
    print(f"Error... Fetching key")

API key has been configured... Successfully AIzaSyCOvx


In [4]:
# Creating client for our LLM model
try:
    client = OpenAI(
        api_key=api_key,
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    print("Client has been created successfully...")
except Exception as e:
    print(f"Error while creating Client: {e}")

Client has been created successfully...


### Using openAI without function calling

In [30]:
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. \
He is Asian American and has a 3.8 GPA. \
David is known for his programming skills and is an active member of the university's Robotics Club. \
He hopes to pursue a career in artificial intelligence after graduating.\
Return only the JSON object, nothing else."
student_1_description

"David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating.Return only the JSON object, nothing else."

In [31]:
prompt_1 = f"""
Please extract the following information about the student from the given text and return the following information as given below:
name
major
school
grades
club
This is the body of text to extract the information from:
{student_1_description}
"""
prompt_1

"\nPlease extract the following information about the student from the given text and return the following information as given below:\nname\nmajor\nschool\ngrades\nclub\nThis is the body of text to extract the information from:\nDavid Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating.Return only the JSON object, nothing else.\n"

In [32]:
# Now lets see the response from our gemini model
response = client.chat.completions.create(
    model='gemini-2.0-flash',
    messages= [{'role':'user', 'content':prompt_1}]
)

print(response.choices[0].message.content)

# From the below response everything looks fine and working properly.
# Now lets change the prompt with something little bit different

```json
{
  "name": "David Nguyen",
  "major": "computer science",
  "school": "Stanford University",
  "grades": "3.8 GPA",
  "club": "Robotics Club"
}
```


In [33]:
student_2_description="Ravi Patel is a sophomore majoring in computer science at the University of Michigan. \
He is South Asian Indian American and has a 3.7 GPA. \
Ravi is an active member of the university's Chess Club and the South Asian Student Association. \
He hopes to pursue a career in software engineering after graduating.\
Return only the JSON object, nothing else."

prompt_2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grade
club

This is the body of text to extract the information from:
{student_2_description}
'''

In [34]:
# Now let's see the response for the 2nd prompt
response = client.chat.completions.create(
    model='gemini-2.0-flash',
    messages= [{'role':'user', 'content':prompt_2}]
)

print(response.choices[0].message.content)

# As we can see, it is not consistent. Instead of returning one club, it has returned the list of clubs joined by Ravi. It is also different from the first student.
# If we run the response couple of time it's not consistent sometimes it's shows single club and sometimes both of them 
# To make the response consistant we will use tools/function.

```json
{
  "name": "Ravi Patel",
  "major": "computer science",
  "school": "University of Michigan",
  "grade": "sophomore",
  "club": "Chess Club"
}
```


## OpenAI function calling example

In [35]:
# Creating a fucntion for extracting studnet info
def extract_student_info(name, major, school, grades, club):
    return f"{name} is majoring in {major} at {school}. He has {grades} GPA and he is an active member of university's {club}."

def extract_school_info(name, ranking, country, no_of_students):
    return f"{name} is located in the {country}. The university is ranked #{ranking} in the world with {no_of_students} students."

In [36]:
student_custom_functions = [
    {
        'name':'extract_student_info',
        'description':"Get the student information from the body of the input text.",
        'parameter':{
            'name':{
                'type':'string',
                'description':'Name of the person'
            },
            'major':{
                'type':'string',
                'description':'Major subject'
            },
            'school':{
                'type':'string',
                'description':'The university name.'
            },
            'grades':{
                'type':'string',
                'description':'GPA of student'
            },
            'club':{
                'type':'string',
                'description':'School club for extracurricular activities.'
            }
        }
    }
]

In [42]:
import json
student_description = [student_1_description,student_2_description]
for i in student_description:
    response = client.chat.completions.create(
        model = 'gemini-2.0-flash',
        messages = [{'role': 'user', 'content': i}],
        # functions = student_custom_functions,
        # function_call = 'auto'
    )
    # Extract the content from the response
    content = response.choices[0].message.content
    
    # Clean the content to ensure it's valid JSON
    # Remove any leading/trailing whitespace and quotes
    content = content.strip().strip('"')
    
    # If the content starts with a newline, remove it
    if content.startswith('\n'):
        content = content.lstrip()
    
    # If the content is wrapped in quotes, remove them
    if content.startswith('"') and content.endswith('"'):
        content = content.strip('"')

    # Loading the response as a JSON object
    try:
        json_response = json.loads(content)  # Use the cleaned content instead of raw response
        print(json_response)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        print(f"Raw content: {content}")

Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Raw content: ```json
{
  "name": "David Nguyen",
  "year": "Sophomore",
  "major": "Computer Science",
  "university": "Stanford University",
  "ethnicity": "Asian American",
  "gpa": 3.8,
  "skills": ["Programming"],
  "activities": ["Robotics Club"],
  "career_goals": "Artificial Intelligence"
}
```
Error decoding JSON: Expecting value: line 1 column 1 (char 0)
Raw content: ```json
{
  "name": "Ravi Patel",
  "year": "Sophomore",
  "major": "Computer Science",
  "university": "University of Michigan",
  "ethnicity": "South Asian Indian American",
  "gpa": 3.7,
  "activities": [
    "Chess Club",
    "South Asian Student Association"
  ],
  "career_goal": "Software Engineering"
}
```
